In [ ]:
import pandas as pd
from datetime import datetime
def is_in_schedule(schedule, datetime_obj):
    # Get the day of the week from the datetime object
    day_of_week = datetime_obj.strftime("%A")

    # Check if the day is in the schedule
    if day_of_week not in schedule:
        return True 

    if schedule[day_of_week] == 0:
        return False

    # Get the intervals for the day from the schedule
    intervals = schedule[day_of_week]
    intervals = intervals.strip('[]').split(', ')
    print(intervals)
    intervals_fix = []
    for interval in intervals:
        splits = interval.split('-')
        interval_lst = []
        for element in splits:
            interval_lst.append(element.strip())
        intervals_fix.append(interval_lst)
    print(intervals_fix)

    for start, end in intervals_fix:
        pattern = r'^\d{2}'
    
        # Use the search function to find if the pattern matches the string
        match_start = re.search(pattern, start)
        match_end = re.search(pattern, end)
        if bool(match_start) == True:
            if int(start[:2]) > 12:
                new_num = int(start[:2]) - 12
                start = str(new_num) + start[2:]
        if bool(match_end) == True:
            if int(end[:2]) > 12:
                new_num = int(end[:2]) - 12
                end = str(new_num) + end[2:]
    
        try:
            start_time = datetime.strptime(start, '%I %p').time()
        except ValueError:
            start_time = datetime.strptime(start, '%I:%M %p').time()

        try:
            end_time = datetime.strptime(end, '%I %p').time()
            
        except ValueError:
            end_time = datetime.strptime(end, '%I:%M %p').time()        

        # Check if the datetime falls within this interval
        if start_time <= datetime_obj.time() <= end_time:
            return True

    # If none of the intervals match, return False
    return False
            

all_aeds = pd.read_excel("C:\Users\jorge\Downloads\aed_lat_long_availability.xlsx")
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]
start_index = 15227

# Select the last 1000 rows starting from the defined index until the end and swap latitude and longitude columns within that subset
last_1000_rows = all_aeds.iloc[start_index:]
last_1000_rows['latitude'], last_1000_rows['longitude'] = last_1000_rows['longitude'], last_1000_rows['latitude']

# Update the last 1000 rows in the original DataFrame
all_aeds.iloc[start_index:] = last_1000_rows

all_aeds = all_aeds.dropna(subset=['longitude'])

# Reset index
all_aeds = all_aeds.reset_index(drop=True)

for i in range(0, len(all_aeds)):
    for column in weekdays:
        if type(all_aeds[column][i]) == str:
            if all_aeds[column][i][-1] != ']':
                new_val = all_aeds[column][i].strip()
                all_aeds[column][i] = new_val

# Example usage:
maps = MapAEDS()
for i in range(0, len(all_aeds)):
    dict_schedule = {}
    for weekday in weekdays:
        if isinstance(all_aeds[weekday][i] , str) or all_aeds[weekday][i] == 0:
            dict_schedule[weekday] = all_aeds[weekday][i]            
    point = AEDPoint(i, all_aeds['latitude'][i], all_aeds['longitude'][i], all_aeds['address'][i], all_aeds['location'][i], dict_schedule, False)
    maps.add_point(point, True)
    
current_time = datetime.now()

for point in maps.points:
    if maps.points[point][0].taken == False:
        maps.update_availability(point, 1, time = current_time)

nearby_points = maps.points_within_radius(user_loc, 10.8)